In [36]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import random

# Parameters Config

In [37]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [38]:
TRAINING_SIZE = 100000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789-+ '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [39]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [40]:
ctable = CharacterTable(chars)

In [41]:
ctable.indices_char
#ctable.char_indices
#ctable.chars

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [42]:
questions = []
expected = []
seen = set()
countAdd = 0
countSub = 0
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b),reverse=True))  
    if key in seen:
        continue
    seen.add(key)
    ranAddSub = random.randint(0,1)
    #print (ranAddSub)
    if ranAddSub==0:
        countAdd+=1
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        ans += ' ' * (DIGITS + 1 - len(ans))
    else :
        countSub+=1
        a,b = sorted((a, b),reverse=True)
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
        ans += ' ' * (DIGITS + 1 - len(ans))
    #print(query)
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))
print(countAdd , countSub)

Generating data...
Total addition questions: 100000
50307 49693


In [43]:
print(questions[:5], expected[:5])

['8+84   ', '21-0   ', '718+88 ', '163+842', '56-11  '] ['92  ', '21  ', '806 ', '1005', '45  ']


# Processing

In [44]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [45]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:80000]
train_y = y[:80000]
test_x = x[80000:]
test_y = y[80000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(72000, 7, 13)
(72000, 4, 13)
Validation Data:
(8000, 7, 13)
(8000, 4, 13)
Testing Data:
(20000, 7, 13)
(20000, 4, 13)


In [46]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False False]
  [False False False False False False False False False False False
    True False]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False  True False False False False
   False False]
  [False False False False  True False False False False False False
   False False]
  [False False False False False False  True False False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False False False False False
    True False]
  [False False False False False False False False False False False
    True False]
  [False False False False False False False False False  True False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False False False False Fals

# Build Model

In [47]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [48]:
for iteration in range(40):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 25s 353us/step - loss: 1.8396 - acc: 0.3293 - val_loss: 1.7579 - val_acc: 0.3449
Q 545-58  T 487  ☒ 328 
Q 811-560 T 251  ☒ 22  
Q 103-57  T 46   ☒ 22  
Q 306-280 T 26   ☒ 228 
Q 642-83  T 559  ☒ 228 
Q 956-16  T 940  ☒ 688 
Q 6+563   T 569  ☒ 888 
Q 521-65  T 456  ☒ 22  
Q 172+184 T 356  ☒ 112 
Q 642+3   T 645  ☒ 688 

--------------------------------------------------
Iteration 1
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 24s 336us/step - loss: 1.6812 - acc: 0.3723 - val_loss: 1.6008 - val_acc: 0.4017
Q 268+81  T 349  ☒ 797 
Q 128-65  T 63   ☒ 11  
Q 60+299  T 359  ☒ 606 
Q 502-91  T 411  ☒ 366 
Q 551-44  T 507  ☒ 566 
Q 829-322 T 507  ☒ 101 
Q 75+785  T 860  ☒ 900 
Q 440+31  T 471  ☒ 457 
Q 84+706  T 790  ☒ 807 
Q 274-99  T 175  ☒ 11  

-----------------

KeyboardInterrupt: 

# Testing

In [31]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
equalAnswer = 0
predictions = model.predict_classes(test_x)

for i in range(len(test_x)):
    predAnswer = ctable.decode(predictions[i],False)
    correctAnswer = ctable.decode(test_y[i])
    if(predAnswer == correctAnswer):
        equalAnswer+=1        

accuracyAnswer = equalAnswer/len(test_y)
accuracyAnswer

MSG : Prediction


0.85475